In [11]:
%store -r n_user
%store -r n_item

print(n_user)
print(n_item)

2979
2999


In [12]:
# import requirements
import os
import json
import sagemaker
import numpy as np
import pandas as pd
import tensorflow as tf
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlow

# get current SageMaker session's execution role and default bucket name
sagemaker_session = sagemaker.Session()

role = get_execution_role()
print("execution role ARN:", role)

# bucket_name_default = sagemaker_session.default_bucket()
# print(bucket_name_default)
# bucket_name = 'ml-demo-common'
%store -r bucket_name
print(bucket_name)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
execution role ARN: arn:aws:iam::629515838455:role/SageMaker_smwu
sagemaker-ap-northeast-2-629515838455


In [13]:
# specify the location of the training data
training_data_uri = os.path.join(f's3://{bucket_name}', 'data')
print(training_data_uri)

s3://sagemaker-ap-northeast-2-629515838455/data


In [14]:
# inspect the training script using `pygmentize` magic
!pygmentize 'ncf.py'

"""

 Copyright Amazon.com, Inc. or its affiliates. All Rights Reserved.
 SPDX-License-Identifier: MIT-0
 
 Permission is hereby granted, free of charge, to any person obtaining a copy of this
 software and associated documentation files (the "Software"), to deal in the Software
 without restriction, including without limitation the rights to use, copy, modify,
 merge, publish, distribute, sublicense, and/or sell copies of the Software, and to
 permit persons to whom the Software is furnished to do so.

 THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED,
 INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A
 PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT
 HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION
 OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE
 SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

"""


import tensorflow

In [15]:
# specify training instance type and model hyperparameters
# note that for the demo purpose, the number of epoch is set to 1

num_of_instance = 1                 # number of instance to use for training
instance_type = 'ml.m5.large'     # type of instance to use for training

training_script = 'ncf.py'

training_parameters = {
    'epochs': 1,
    'batch_size': 256, 
    'n_user': n_user, 
    'n_item': n_item
}

# training framework specs
tensorflow_version = '2.1.0'
python_version = 'py3'
distributed_training_spec = {'parameter_server': {'enabled': True}}

In [16]:
# initiate the training job using Tensorflow estimator
ncf_estimator = TensorFlow(
    entry_point=training_script,
    role=role,
    instance_count=num_of_instance,
    instance_type=instance_type,
    framework_version=tensorflow_version,
    py_version=python_version,
    distribution=distributed_training_spec,
    hyperparameters=training_parameters
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [17]:
# kick off the training job
ncf_estimator.fit(training_data_uri)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Amazon SageMaker Debugger does not currently support Parameter Server distribution
INFO:sagemaker:Amazon SageMaker Debugger does not currently support Parameter Server distribution
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: tensorflow-training-2023-12-05-05-14-52-945


Using provided s3_resource
2023-12-05 05:14:53 Starting - Starting the training job...
2023-12-05 05:15:10 Starting - Preparing the instances for training......
2023-12-05 05:16:15 Downloading - Downloading input data.........
2023-12-05 05:17:41 Training - Training image download completed. Training in progress.2023-12-05 05:17:44,330 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2023-12-05 05:17:44,338 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-12-05 05:17:44,556 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-12-05 05:17:44,573 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-12-05 05:17:44,588 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2023-12-05 05:17:44,599 sagemaker-containers INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {
        "sagemaker_parameter_server_e


2023-12-05 05:18:38 Uploading - Uploading generated training model
2023-12-05 05:18:38 Completed - Training job completed
Training seconds: 142
Billable seconds: 142


In [20]:
endpoint_name = 'ep-rest-rec-002'
model_name = 'rest-recommend-model-sg02'

predictor = ncf_estimator.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.large", 
    endpoint_name=endpoint_name,
    model_name=model_name,
)

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating model with name: rest-recommend-model-sg02
INFO:sagemaker:Creating endpoint-config with name ep-rest-rec-002
INFO:sagemaker:Creating endpoint with name ep-rest-rec-002


---!

In [21]:
print(f'endpoint_name= "{endpoint_name}"')
print(f'bucket_name = "{bucket_name}"')

endpoint_name= "ep-rest-rec-002"
bucket_name = "sagemaker-ap-northeast-2-629515838455"


In [11]:
# delete endpoint at the end of the demo
predictor.delete_endpoint(delete_endpoint_config=True)

INFO:sagemaker:Deleting endpoint configuration with name: ep-rest-rec-sg02
INFO:sagemaker:Deleting endpoint with name: ep-rest-rec-sg02
